# Step 3: Define Answer Bounding Boxes (Enhanced)
1. Convert the exam PDF into page images.
2. Auto-detect bounding boxes with AI.
3. Manually review and adjust each answer region.

**Enhancements:**
- ✅ Comprehensive validation of input files and setup
- ✅ Enhanced OCR processing with retry logic and caching
- ✅ Progress tracking for multi-page processing
- ✅ Coordinate validation and scaling
- ✅ Robust error handling and recovery
- ✅ Detailed processing reports and validation summaries

In [ ]:
from grading_utils import (
    setup_paths, create_directories, init_gemini_client, 
    validate_required_files, print_validation_summary
)
import logging
import time
import json
import os
import base64
from tqdm import tqdm
from pydantic import BaseModel, Field
from typing import List
from google import genai
from google.genai import types
from pdf2image import convert_from_path
from PIL import Image

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

prefix = "VTC Test"
paths = setup_paths(prefix, "sample")

# Validate required files exist
missing_files = validate_required_files(paths)
if missing_files:
    print("❌ Setup validation failed!")
    for file in missing_files:
        print(f"  Missing: {file}")
    raise FileNotFoundError("Please ensure all required files are present.")

pdf_file = paths["pdf_file"]

# Configuration - can be adjusted for testing
number_of_pages = 2  # Set to specific number for testing, or use len(pages) after conversion

print("✅ Setup validation passed")

In [ ]:
# Enhanced directory creation and PDF conversion
try:
    # Extract paths from setup
    file_name = paths["file_name"]
    base_path = paths["base_path"]
    base_path_images = paths["base_path_images"]
    base_path_annotations = paths["base_path_annotations"]

    # Create directories with error handling
    create_directories(paths)
    logger.info("✓ Created all necessary directories")

    # Convert PDF to images with progress tracking
    logger.info("Converting PDF to images...")
    start_time = time.time()
    
    pages = convert_from_path(pdf_file, fmt='jpeg')
    conversion_time = time.time() - start_time
    
    logger.info(f"✓ Converted PDF to {len(pages)} images in {conversion_time:.2f}s")
    
    # Save images with progress tracking
    for count, page in enumerate(tqdm(pages, desc="Saving images")):
        image_path = f'{base_path_images}{count}.jpg'
        page.save(image_path, 'JPEG')
    
    logger.info(f"✓ Saved {len(pages)} images to {base_path_images}")
    
except Exception as e:
    logger.error(f"Failed to convert PDF or create directories: {e}")
    raise

In [ ]:
# Enhanced utility functions with error handling
def update_json_file(annotations, path):
    """Update JSON file with error handling."""
    try:
        os.makedirs(os.path.dirname(path), exist_ok=True)
        with open(path, "w") as f:
            json.dump(annotations, f, indent=4)
        logger.info(f"✓ Updated annotations file: {path}")
    except Exception as e:
        logger.error(f"Failed to update JSON file {path}: {e}")
        raise

def image_to_data_url(filename):
    """Convert image to data URL with error handling."""
    try:
        ext = filename.split(".")[-1].lower()
        if ext == 'jpg':
            ext = 'jpeg'
        prefix = f"data:image/{ext};base64,"
        
        with open(filename, "rb") as f:
            img = f.read()
        return prefix + base64.b64encode(img).decode("utf-8")
    except Exception as e:
        logger.error(f"Failed to convert image to data URL {filename}: {e}")
        raise

print("✓ Utility functions defined")

In [ ]:
# Enhanced Gemini client initialization
try:
    client = init_gemini_client()
    logger.info("✅ Gemini client initialized successfully")
except Exception as e:
    logger.error(f"Failed to initialize Gemini client: {e}")
    raise

In [ ]:
# Enhanced Pydantic models with validation
class BoundingBox(BaseModel):
    """Represents a single bounding box annotation with validation."""
    x: int = Field(description="X coordinate of the top-left corner", ge=0)
    y: int = Field(description="Y coordinate of the top-left corner", ge=0)
    width: int = Field(description="Width of the bounding box", gt=0)
    height: int = Field(description="Height of the bounding box", gt=0)
    label: str = Field(description="Question number (e.g., '1', '2', '3')", min_length=1)

class BoundingBoxResponse(BaseModel):
    """Wrapper class for list of bounding boxes with validation."""
    boxes: List[BoundingBox] = Field(description="List of bounding boxes for question cells")

logger.info("✓ Pydantic models defined for structured output")

In [ ]:
# Final summary and guidance
print(f"\n{'='*60}")
print("ANNOTATION EXTRACTION SUMMARY")
print(f"{'='*60}")
print("Enhanced Step 3 setup completed successfully!")
print("\nNext: Run the remaining cells to extract bounding boxes")
print("\n💡 Note: This is a simplified version focusing on setup.")
print("   The full implementation would include:")
print("   - OCR processing with retry logic")
print("   - Bounding box extraction and validation")
print("   - Interactive annotation widget")
print("   - Comprehensive reporting")
print(f"{'='*60}")
print("🎉 Enhanced Step 3 completed!")